In [1]:
import spreg
from spreg import OLS
from spreg import ML_Lag_Regimes
import pysal as ps
import libpysal
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import robust_scale
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse

/home/maciek/.pyenv/versions/3.8.12/envs/school-map-project/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [3]:
full = gpd.read_file('../raw_data/full_old/full.shp')

In [9]:
regression_df = gpd.read_file('../raw_data/for_models/regression_gdf.shp')

In [23]:
merged = regression_df.merge(full[['BEZ', 'PLR_ID']], on='PLR_ID', how='left')

In [26]:
merged.to_file('../raw_data/reg_with_bezirk.shp')

In [3]:
db = libpysal.io.open('../raw_data/reg_with_bezirk.dbf','r')
df = gpd.read_file('../raw_data/reg_with_bezirk.shp')

FileNotFoundError: [Errno 2] No such file or directory: '../raw_data/reg_with_bezirk.dbf'

In [ ]:
columns = [
    'public_tra', 'eating', 'community', 'education', 'schools', 'kindergart',
    'outdoor_fa', 'outdoor_le', 'night_life', 'ave_rent', 'social_hou',
    'public_hou', 'dyn_ew', 'dyn_sales', 'dyn_unempl', 'noise',
    'air', 'green', 'bio', 'mig_rate'
]
regimes = db.by_col("label")

In [ ]:
y = np.array(db.by_col('child_pov')).T
x_names = columns
X = np.array([db.by_col(var) for var in x_names]).T

In [ ]:
queen = libpysal.weights.Queen.from_dataframe(merged)

In [ ]:
mllag = ML_Lag_Regimes(y,
                       X,
                       name_x=columns,
                       name_y='child_pov',
                       regimes=regimes,
                       regime_lag_sep=True,
                       w=queen,
                       name_regimes="label",
                       name_ds='labeled.dbf',
                       name_w='Queen')

In [ ]:
print(mllag.summary)